In [1]:
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure
import json
import matplotlib.pyplot as plt

In [6]:
# 设置目标图像的索引
target = 2
data_name = ['0618', '0854', '1066'][target - 1]

# 读取RGB图像
img = cv2.imread(f'../input_data/{data_name}.png')
print(f"Image Shape: {img.shape}")

# 假设你已经准备好了标签
labelme_json_path = f'../input_data/{data_name}.json'

# 读取LabelMe的JSON文件
with open(labelme_json_path, 'r') as f:
    label_data = json.load(f)

# 转换图像到RGB格式（OpenCV使用BGR格式）
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# HOG特征提取 - 分通道提取
def extract_hog_features_rgb(img_rgb):
    hog_features = []
    
    # 为每个颜色通道提取HOG特征
    for i in range(3):  # 对于R, G, B通道
        channel = img_rgb[:, :, i]
        fd, _ = hog(channel, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
        hog_features.append(fd)
    
    # 合并三个通道的特征
    return np.concatenate(hog_features, axis=0)

# 提取HOG特征
hog_features = extract_hog_features_rgb(img_rgb)
print(f"HOG Features Shape: {hog_features.shape}")

# 保存HOG特征
np.save(f'./Features/{data_name}_hog_features.npy', hog_features)

# ---- 添加标签 ----
# 创建一个与图像大小相同的空白标签图，初始值为0
label_image = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)

# 遍历每个标注对象并绘制标签
for shape in label_data['shapes']:
    label_name = shape['label']  # 获取标签名称
    points = np.array(shape['points'], dtype=np.int32)  # 获取多边形的顶点

    # 绘制多边形区域为标签
    if label_name == 'road':  # 例如，'road'对应标签1
        cv2.fillPoly(label_image, [points], 1)
    elif label_name == 'building':  # 例如，'building'对应标签2
        cv2.fillPoly(label_image, [points], 2)

# 保存标签图像
np.save(f'./Features/{data_name}_hog_labels.npy', label_image)

print(f"Labels saved as {data_name}_labels.npy")

# 打印标签图像的部分内容进行验证
print(label_image.shape)

Image Shape: (250, 500, 3)
HOG Features Shape: (197640,)
Labels saved as 0854_labels.npy
(250, 500)
